In [0]:
# %pip install databricks-vectorsearch
# dbutils.library.restartPython()

In [0]:
table_properties = spark.sql("SHOW TBLPROPERTIES amitabh_arora_catalog.playground.customer_names")
change_data_feed_enabled = table_properties.filter(table_properties.key == 'delta.enableChangeDataFeed').select('value').collect()[0][0]
display(change_data_feed_enabled)

In [0]:
from databricks.vector_search.client import VectorSearchClient

client = VectorSearchClient()

### Check if vector index already exists

In [0]:
index_exists = False
try:
    index = client.get_index(endpoint_name="one-env-shared-endpoint-17", index_name="amitabh_arora_catalog.playground.customer_names_index")
    index_exists = True
    print("Index exists")
except Exception as e:
    print(f"Index DOES NOT exist")


### Create the vector search index if it does not already exist, else sync it

In [0]:
if not index_exists:
    print("Creating index...")
    index = client.create_delta_sync_index(
        endpoint_name="one-env-shared-endpoint-17",
        source_table_name="amitabh_arora_catalog.playground.customer_names",
        index_name="amitabh_arora_catalog.playground.customer_names_index",
        pipeline_type="TRIGGERED",
        primary_key="id",
        embedding_source_column="cust_name",
        embedding_model_endpoint_name="databricks-gte-large-en")
else:
    print("SYNC'ing index...")
    index.sync()